# A class RangeSumArray with the following functions:

1. sumRange(i,j), return the sum from the element i to element j in the array

2. update(i,val), updates the element i in the array to val

# Examples

A = RangeSumArray([1,3,5])

A.sumRange(0, 2) -> 9

A.update(1, 2)

A.sumRange(1, 2) -> 7

# Binary indexed tree solution

Best explanation so far: https://cs.stackexchange.com/questions/10538/bit-what-is-the-intuition-behind-a-binary-indexed-tree-and-how-was-it-thought-a

### Construct a BI Tree (O(nlogn))
BITree[7] = A[6]

BITree[8] = A[7] + BITree[7] + BITree[6] + BITree[4]

BITree[6] = A[5] + BITree[5]

BITree[k] = A[k-1] + BITree[k-1] + BITree[k-2] + ... + BITree[k-2^m] where 2^m|k

In [1]:
def foo(x):
    l=[]
    t=1
    while x%(2*t)==0:
        l.append(x-t)
        t = t*2
    return l

In [2]:
print(foo(16),foo(9),foo(12))

[15, 14, 12, 8] [] [11, 10]


## sumRange (O(logn))
sumRange(0,k-1) = BITree[k] + BITree[k->parent] + ... + BITree[k->parent->/...->parent]

![sumRangeTree](sumRangeTree.png)

In [3]:
def get_parents(x):
    l=[]
    while x>0:
        l.append(x)
        x = x - (x&(-x))
    return l

In [5]:
get_parents(15)

[15, 14, 12, 8]

## Update (O(logn))
update(k-1,val)

Update
BITree[k], BITree[k->parent], ..., BITree[k->parent->...->parent]


![updateTree](updateTree.png)

In [6]:
def get_parents(x,M):
    l=[]
    while x<M:
        l.append(x)
        x = x + (x&(-x))
    return l

In [7]:
get_parents(3,16)

[3, 4, 8]

In [8]:
class RangeSumArray():
    def __init__(self, array):
        """
        :type nums: List[int]
        """
        self.array = array
        self.BITree = [0]
        
        for i in range(len(array)):
            s = array[i]
            t=1
            while (i+1)%(2*t)==0:
                s += self.BITree[i+1-t]
                t = t*2
            self.BITree.append(s)

    def cumSum(self, i):
        """
        :type i: int
        """
        s = 0
        i = i+1
        while i>0:
            s += self.BITree[i]
            i = i - (i&(-i))
        return s
        
    def sumRange(self, i, j):
        """
        :type i: int
        :type j: int
        :rtype: int
        """
        if (j<i) or (j<0) or (i>=len(self.array)):
            return 0
        else:
            i = max(0,i)
            j = min(len(self.array)-1,j)
            if i==0:
                return self.cumSum(j)
            else:
                return self.cumSum(j) - self.cumSum(i-1)
        
        
    def update(self, i, val):
        """
        :type i: int
        :type val: int
        :rtype: None
        """
        diff = val - self.array[i]
        self.array[i] = val
        
        i = i+1
        while i<len(self.BITree):
            self.BITree[i] += diff
            i = i + (i&(-i))
            
    def append(self, val):
        pos = len(self.array)
        self.array.append(val)
        
        s = val
        t = 1
        pos = len(self.array)
        while (pos%(t*2)==0):
            s += self.BITree[pos-t]
            t *= 2
        self.BITree.append(s)

In [9]:
a = RangeSumArray([1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1])

In [10]:
a.BITree

[0, 1, 2, 1, 4, 1, 2, 1, 8, 1, 2, 1, 4, 1, 2, 1, 16, 1]

In [11]:
a.sumRange(-10,6)

7

In [12]:
a.update(3,2)

In [13]:
a.sumRange(-10,6)

8

In [14]:
a.array

[1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [15]:
a.BITree

[0, 1, 2, 1, 5, 1, 2, 1, 9, 1, 2, 1, 4, 1, 2, 1, 17, 1]

In [16]:
a.append(3)

In [17]:
a.append(5)
a.append(2)

In [18]:
a.BITree

[0, 1, 2, 1, 5, 1, 2, 1, 9, 1, 2, 1, 4, 1, 2, 1, 17, 1, 4, 5, 11]

In [19]:
a.array

[1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 5, 2]

In [20]:
a.sumRange(10,18)

15

# Segmentation tree Solution

In [21]:
import numpy as np
class RangeSumArray():
    def __init__(self, array):
        """
        :type nums: List[int]
        """
        self.array = array
        n = len(array)
        num_leaf = pow(2,int(np.ceil(np.log2(n))))
        total = num_leaf*2
        self.SegTree = [0 for i in range(total)]
        
        for i in range(n):
            self.SegTree[i+num_leaf] = array[i]
        i = num_leaf - 1
        while i>0:
            self.SegTree[i] = self.SegTree[i*2] + self.SegTree[i*2+1]
            i -= 1
        self.SegTree[0] = num_leaf

    def cumSum(self, i):
        """
        :type i: int
        """
        if i<=0:
            return 0
        if i>len(self.array):
            return self.SegTree[1]
        else:
            right_i = self.SegTree[0]
            left_i = 1
            res = 0
            index = 1
            while True:
                if right_i == i:
                    res += self.SegTree[index]
                    break
                elif right_i > i:
                    right_i = int((right_i+left_i)/2)
                    index *= 2
                else:
                    res += self.SegTree[index]
                    index += 1
                    [left_i, right_i] = [right_i+1, 2*right_i-left_i+1]
                
        
        return res
        
    def sumRange(self, L, R):
        """
        :type i: int
        :type j: int
        :rtype: int
        """
        l = L + self.SegTree[0]
        r = R + self.SegTree[0]
        res = 0
        
        while (l<=r):
            if (l%2 == 1):                # if l is right child of its parent P
                res += self.SegTree[l]    # Don't go up to its parents and add the node value immediately
                l += 1                    # Go to the branch right next to it
                
            if (r%2 == 0):                # if r is left child of its parent P
                res += self.SegTree[r]    # Don't go up to its parents and add the node value immediately
                r -= 1                    # Go to the branch left next to it
                
            # Go up one level
            l = int(l/2)
            r = int(r/2)
        
        return res
        
        
    def update(self, i, val):
        """
        :type i: int
        :type val: int
        :rtype: None
        """
        diff = val - self.array[i]
        self.array[i] = val
        
        pos = i + self.SegTree[0]
        while pos>0:
            self.SegTree[pos] += diff
            pos = int(pos/2)

In [22]:
A = [2,3,5,1,4,6,2,3,4,1,1,1,1,1,1,1,1,1,1,1]

In [23]:
test = RangeSumArray(A) 

In [24]:
for i in range(20):
    print(test.sumRange(10,i))

0
0
0
0
0
0
0
0
0
0
1
2
3
4
5
6
7
8
9
10


In [71]:
test.update(2,2)

In [72]:
test.array

[2, 3, 2, 1, 4, 6, 2, 3, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [77]:
test.cumSum(3)

7

In [29]:
{1:1, 2:2} == {2:2, 1:1}

True

In [32]:
'fjai'.isnumeric()

False

In [37]:
try:
    a = int('faa',16)
except:
    print('wo shi sb')
    

In [38]:
a

4010